The choices for each index column are listed below:
- benchmark: 
    - CIFAR10
    - Imagenet100
- data:
    - ID
    - OoD
- dataset
    - CIFAR10, Textures, SVHN, LSUN-C, LSUN-R, iSUN, Places365, average
    - Imagenet100, NINCO, Textures, iNaturalist, SUN, Places, average
- model: 
    - wrn_40_2
    - resnet50
- variant: 
    - NT
    - DA
    - AT
    - PAT
- detector:
    - Entropy
    - ViM
    - Mahalanobis+ODIN
    - Mahalanobis
    - KLMatching
    - SHE
    - MSP
    - EnergyBased
    - MaxLogit
    - ODIN
    - DICE
    - RMD
- perturb_function:
    - rotation
    - translation
    - scale
    - hue
    - saturation
    - bright_contrast
    - blur  
    - Linf
    - L2
    - average
- severity: 1, 2, 3, 4, 5, average

In [4]:
import pandas as pd
import os
import yaml

# Load configs: benchmarks, model variants, OoD datasets and save directory.
with open('../config.yaml', 'r') as f:
    configs = yaml.safe_load(f)
sorters = dict(benchmark=["CIFAR10", "Imagenet100"], model=["wrn_40_2", "resnet50"],
                    variant=["NT", "DA", "AT", "PAT"], detector=configs["score_functions"],
                    data=["ID", "OoD"], 
                    perturb_function=["average"]+configs["perturb_functions"])


# Read the DAE summary file
filepath = os.path.join("../results/eval/robustness", "dae_summary.csv")
df_dae = pd.read_csv(filepath).copy()
# display(df_dae)

# Read the model accuracy and FPR95 summary file
filepath = os.path.join("../results/eval/performance", "model_performance.csv")
df_mdl = pd.read_csv(filepath).copy()
filepath = os.path.join("../results/eval/performance", "detector_performance.csv")
df_detector = pd.read_csv(filepath).copy()
df_detector = df_detector.query("dataset=='average'").drop(columns=["dataset", "AUROC", "AUPR_IN", "AUPR_OUT"]).\
    rename(columns={"score_function": "detector"}).copy()

### 1. Overview of the DAE rate

In [5]:
# Overview of the DAE rate
# Search for related experiment results
df_filtered = df_dae[
                #  (df_dae["variant"].isin(["NT", "DA"])) &
                #  (df_dae["detector"]=="ODIN") &
                #  (df_dae["data"]=="OoD") &
                 (df_dae["perturb_function"]=="average") &
                 (df_dae["dataset"].isin(["average", "CIFAR10", "Imagenet100"]))
                ].drop(["dataset", "perturb_function"], axis=1).copy()
df_perf = pd.merge(left=df_mdl, right=df_detector, on=["benchmark", "model", "variant"], how="inner")
df_perf_dae = pd.merge(left=df_perf, right=df_filtered, on=["benchmark", "model", "variant", "detector"], how="inner")
df_rlt = df_perf_dae.pivot(index=["benchmark", "model", "variant", "accuracy", "detector", "FPR95",], 
                           columns=["data"])
df_rlt["ID_DAE"] = df_rlt[("dae_mean", "ID")].astype(str)+u"\u00B1"+ df_rlt[("dae_std", "ID")].astype(str)
df_rlt["OOD_DAE"] = df_rlt[("dae_mean", "OoD")].astype(str)+u"\u00B1"+ df_rlt[("dae_std", "OoD")].astype(str)
df_rlt["MAE"] = df_rlt[("mae_mean", "ID")].astype(str)+u"\u00B1"+ df_rlt[("mae_std", "ID")].astype(str)
df_rlt = df_rlt[["ID_DAE", "OOD_DAE", "MAE"]]
df_rlt.columns = df_rlt.columns.droplevel([1])
df_rlt.set_index("MAE", append=True, inplace=True)
df_rlt = df_rlt.reorder_levels(["benchmark", "model", "variant", "accuracy", "MAE", "detector", "FPR95"]).reset_index()

df_rlt_baseline = df_rlt[df_rlt["detector"]=="MSP"]
df_rlt = df_rlt.sort_values(by="FPR95")
df_rlt = df_rlt.groupby(by=["benchmark", "model", "variant", "accuracy", "MAE"]).head(3)
df_rlt = pd.concat([df_rlt, df_rlt_baseline], axis=0).copy()

for sort_col in ["benchmark", "model", "variant"]:
    df_rlt[sort_col] = df_rlt[sort_col].astype("category")
    df_rlt[sort_col] = df_rlt[sort_col].cat.set_categories(sorters[sort_col], ordered=True)
df_rlt.sort_values(by=["benchmark", "model", "variant", "FPR95"], inplace=True)
df_rlt.set_index(["benchmark", "model", "variant", "accuracy", "MAE", "detector", "FPR95"], inplace=True, drop=True)
display(df_rlt)
# df_rlt.to_csv("DAE_overview_table.csv")

ID_DAE  \
benchmark   model    variant accuracy  MAE           detector         FPR95                      
CIFAR10     wrn_40_2 NT      94.730000 11.175±18.64  Mahalanobis+ODIN 33.253345  14.968±20.136   
                                                     Mahalanobis      33.734307  14.737±20.085   
                                                     EnergyBased      36.141178   9.532±16.022   
                                                     MSP              52.781239   8.645±13.104   
                     DA      95.260000 9.525±17.825  DICE             36.494433   9.188±16.206   
                                                     EnergyBased      37.036160   8.971±15.961   
                                                     MaxLogit         37.159747   8.913±15.683   
                                                     MSP              54.956384   7.261±12.114   
                     AT      95.080000 2.54±7.882    EnergyBased      39.548081    3.752±10.95   
                                                     MaxLogit         40.032133    3.52±10.168   
                                                     DICE             41.194888   3.656±10.513   
                                                     MSP              52.889379    3.185±8.183   
                     PAT     84.950000 7.433±17.501  DICE             64.219286   4.027±10.811   
                                                     MaxLogit         64.310947    4.318±11.79   
                                                     EnergyBased      65.406758   3.687±10.325   
                                                     MSP              75.143413    2.957±7.931   
            resnet50 NT      94.840000 8.127±19.475  ViM              26.485885  10.012±20.741   
                                                     DICE             26.801034   8.837±18.913   
                                                     EnergyBased      27.268608    8.72±18.814   
                                                     MSP              50.495896   6.942±14.147   
                     DA      96.380000 2.797±11.902  EnergyBased      16.985077   4.747±15.496   
                                                     DICE             17.064379   4.725±15.376   
                                                     MaxLogit         17.252850   4.753±15.445   
                                                     MSP              38.024862   3.911±12.898   
                     AT      86.840000 4.946±13.997  ViM              57.747407   5.027±14.076   
                                                     ODIN             59.403483     3.17±10.99   
                                                     MaxLogit         59.925641   3.217±11.168   
                                                     MSP              70.019671   2.929±10.389   
                     PAT     82.410000 7.223±16.992  MaxLogit         66.698250   3.933±11.913   
                                                     ODIN             67.342966   3.849±11.695   
                                                     SHE              68.765255   3.619±11.131   
                                                     MSP              73.786008     3.41±9.994   
Imagenet100 resnet50 NT      88.775510 14.809±28.838 SHE              33.921853  12.583±26.732   
                                                     ODIN             37.891911   13.025±27.01   
                                                     DICE             41.234911  13.297±27.408   
                                                     MSP              50.058787  10.218±22.103   
                     DA      89.915966 12.288±27.472 SHE              35.313137  13.621±28.146   
                                                     MaxLogit         39.022574  13.222±28.219   
                                                     DICE             39.193055  13.861±29.006   
                                                     MSP              

### 2. Show DA improvement from NT

In [ ]:
# Show DA improvement from NT

# Search for related experiment results
df_filtered = df_dae[
                #  (df_dae["variant"].isin(["NT", "DA"])) &
                #  (df_dae["detector"]=="ODIN") &
                #  (df_dae["data"]=="OoD") &
                 (df_dae["perturb_function"]=="average") &
                 (df_dae["dataset"].isin(["average", "CIFAR10", "Imagenet100"]))
                ].copy()

df_filtered.drop(["dataset", "perturb_function", "mae_mean", "mae_std"], axis=1, inplace=True)
for sort_col in ["benchmark", "model", "detector", "variant", "data"]:
    df_filtered[sort_col] = df_filtered[sort_col].astype("category")
    df_filtered[sort_col] = df_filtered[sort_col].cat.set_categories(sorters[sort_col], ordered=True)
df_filtered.sort_values(by=["benchmark", "model", "detector", "variant", "data"], inplace=True)

df_rlt = df_filtered.pivot(index=["benchmark", "model", "detector"], columns=["variant", "data", ])
df_rlt["ID_NT_DAE"] = df_rlt[("dae_mean", "NT", "ID")].astype(str)+u"\u00B1"+ df_rlt[("dae_std", "NT", "ID")].astype(str)
df_rlt["ID_DA_DAE"] = df_rlt[("dae_mean", "DA", "ID")].astype(str)+u"\u00B1"+ df_rlt[("dae_std", "DA", "ID")].astype(str)
df_rlt["OOD_NT_DAE"] = df_rlt[("dae_mean", "NT", "OoD")].astype(str)+u"\u00B1"+ df_rlt[("dae_std", "NT", "OoD")].astype(str)
df_rlt["OOD_DA_DAE"] = df_rlt[("dae_mean", "DA", "OoD")].astype(str)+u"\u00B1"+ df_rlt[("dae_std", "DA", "OoD")].astype(str)
df_rlt = df_rlt[["ID_NT_DAE", "ID_DA_DAE", "OOD_NT_DAE", "OOD_DA_DAE"]]
df_rlt.columns = df_rlt.columns.droplevel([1,2])

def highlighter(x):
    s1 = "color: blue;" if float(x["ID_NT_DAE"].split(u"\u00B1")[0]) < float(x["ID_DA_DAE"].split(u"\u00B1")[0]) else "color: red;"
    s2 = "color: blue;" if float(x["OOD_NT_DAE"].split(u"\u00B1")[0]) < float(x["OOD_DA_DAE"].split(u"\u00B1")[0]) else "color: red;"
    return [""]+[s1] + [""]+[s2]
    
    
df_rlt = df_rlt.style.apply(highlighter, axis=1)
display(df_rlt)
# df_rlt.to_latex("DA_DAE_table.md")

### Overview of DAE rate under 1-5 severity levels

In [ ]:
# Read the DAE summary file
filepath = os.path.join("../results/eval/severity_levels/robustness", "dae_summary.csv")
df_dae = pd.read_csv(filepath).copy()

df_filtered = df_dae[
                #  (df_dae["variant"].isin(["NT", "DA"])) &
                #  (df_dae["detector"]=="ODIN") &
                #  (df_dae["data"]=="OoD") &
                 (df_dae["perturb_function"]=="average") &
                 (df_dae["dataset"].isin(["average", "CIFAR10", "Imagenet100"]))
                ].drop(["dataset", "perturb_function"], axis=1).copy()
df_perf = pd.merge(left=df_mdl, right=df_detector, on=["benchmark", "model", "variant"], how="inner")
df_perf_dae = pd.merge(left=df_perf, right=df_filtered, on=["benchmark", "model", "variant", "detector"], how="inner")
df_rlt = df_perf_dae.pivot(index=["benchmark", "model", "variant", "accuracy", "detector", "FPR95", "severity"], 
                           columns=["data"])

df_rlt["ID_DAE"] = df_rlt[("dae_mean", "ID")].astype(str)+u"\u00B1"+ df_rlt[("dae_std", "ID")].astype(str)
df_rlt["OOD_DAE"] = df_rlt[("dae_mean", "OoD")].astype(str)+u"\u00B1"+ df_rlt[("dae_std", "OoD")].astype(str)
df_rlt["MAE"] = df_rlt[("mae_mean", "ID")].astype(str)+u"\u00B1"+ df_rlt[("mae_std", "ID")].astype(str)
df_rlt = df_rlt[["ID_DAE", "OOD_DAE", "MAE"]]
df_rlt.columns = df_rlt.columns.droplevel([1])
df_rlt.set_index("MAE", append=True, inplace=True)
df_rlt = df_rlt.reorder_levels(["benchmark", "model", "variant", "accuracy", "MAE", "detector", "FPR95", "severity"]).reset_index()

df_rlt_baseline = df_rlt[df_rlt["detector"]=="MSP"]
df_rlt = df_rlt.sort_values(by="FPR95")
df_rlt = df_rlt.groupby(by=["benchmark", "model", "variant", "accuracy", "MAE"]).head(3)
df_rlt = pd.concat([df_rlt, df_rlt_baseline], axis=0).copy()

for sort_col in ["benchmark", "model", "variant"]:
    df_rlt[sort_col] = df_rlt[sort_col].astype("category")
    df_rlt[sort_col] = df_rlt[sort_col].cat.set_categories(sorters[sort_col], ordered=True)
df_rlt.sort_values(by=["benchmark", "model", "variant", "FPR95", "severity"], inplace=True)
df_rlt.set_index(["benchmark", "model", "variant", "accuracy", "detector", "FPR95", "severity", "MAE"], inplace=True, drop=True)
display(df_rlt)
